In [ ]:
pip install catboost

In [ ]:
!pip install transformers

In [ ]:
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re 
from pymystem3 import Mystem

import pandas as pd 
import numpy as np 

from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool
from tqdm import notebook
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import torch
import transformers as ppb # pytorch transformers

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/toxic_comments.csv')
df.head(5)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
df.shape

(159571, 2)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [ ]:
df['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

In [ ]:
batch_1 = df[:5000]

Загрузка предобученной модели DistilBERT и токенизатора

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Хотите BERT вместо distilBERT? Раскомментируйте следующую строку:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Загрузка предобученной модели/токенизатора 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = batch_1['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512)))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
tokenized

0       [101, 7526, 2339, 1996, 10086, 2015, 2081, 210...
1       [101, 1040, 1005, 22091, 2860, 999, 2002, 3503...
2       [101, 4931, 2158, 1010, 1045, 1005, 1049, 2428...
3       [101, 1000, 2062, 1045, 2064, 1005, 1056, 2191...
4       [101, 2017, 1010, 2909, 1010, 2024, 2026, 5394...
                              ...                        
4995    [101, 1000, 7592, 7871, 6820, 2232, 15557, 534...
4996    [101, 1012, 1012, 1012, 2008, 1005, 1055, 2339...
4997    [101, 2053, 1010, 2009, 1005, 1055, 2025, 1037...
4998    [101, 1000, 1037, 7263, 4489, 2007, 2017, 1045...
4999    [101, 1000, 14652, 1012, 1996, 3606, 2003, 205...
Name: text, Length: 5000, dtype: object

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])



In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5000, 512)

In [ ]:
np.array(padded).shape

(5000, 512)

In [ ]:
from tqdm import notebook

In [ ]:
# а теперь сформируем вектора текстов с помощью модели BERT
# это не быстрый процесс(54:27), импортируем инструмент для визуализации времени обработки в цикле
from tqdm import notebook

# для того, чтобы модель отработала в условиях ограниченных ресурсов - оперативной памяти, мы разделяем входной датасет на батчи.
# при батче в 100 твитов потребление оперативной памяти укладывается в 1Гб
batch_size = 50

# Делаем пустой список для хранения эмбеддингов (векторов) твитов
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        # преобразуем батч с токенизированными твитами в тензор 
        # по сути тензор - это многомерный массив, который может быть обработан нейронной сетью
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda() 
        # создаем тензор и для подготовленной маски
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
        
        # передаем в модель BERT тензор из твитов и маску - на выходе получаем эмбеддинги - вектор текста твита
        # torch.no_grad() - для ускорения инференса модели отключим рассчет градиентов
        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

            embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
            del batch
            del attention_mask_batch
            del batch_embeddings


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# преобразуем список батчей эмбеддингов в numpy-матрицу 
features = np.concatenate(embeddings)

In [ ]:
features.shape

(5000, 768)

In [ ]:
labels = batch_1['toxic']

In [ ]:
labels.shape

(5000,)

In [ ]:
labels

0         0
1         0
2         0
3         0
4         0
         ..
159566    0
159567    0
159568    0
159569    0
159570    0
Name: toxic, Length: 159571, dtype: int64

In [ ]:
features_train,features_test,target_train,target_test=train_test_split(features,labels,test_size=0.2,random_state=42)

In [ ]:
lr_clf = LogisticRegression(random_state=42)
lr_clf.fit(features_train,target_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=42)

In [ ]:
predict=lr_clf.predict(features_test)
f1_line=f1_score(predict,target_test)
f1_line

0.7200000000000001

In [ ]:
cat=CatBoostClassifier(
    loss_function="Logloss",
    iterations=400,
    learning_rate=0.01
)

In [ ]:
cat.fit(features_train, target_train, verbose=10)

0:	learn: 0.6804176	total: 227ms	remaining: 1m 30s
10:	learn: 0.5719745	total: 2.07s	remaining: 1m 13s
20:	learn: 0.4850460	total: 3.89s	remaining: 1m 10s
30:	learn: 0.4199243	total: 5.71s	remaining: 1m 8s
40:	learn: 0.3689124	total: 7.56s	remaining: 1m 6s
50:	learn: 0.3295208	total: 9.36s	remaining: 1m 4s
60:	learn: 0.2984227	total: 11.2s	remaining: 1m 2s
70:	learn: 0.2719709	total: 13.1s	remaining: 1m
80:	learn: 0.2497752	total: 14.9s	remaining: 58.7s
90:	learn: 0.2328360	total: 16.8s	remaining: 57.2s
100:	learn: 0.2194329	total: 18.7s	remaining: 55.3s
110:	learn: 0.2080507	total: 20.6s	remaining: 53.6s
120:	learn: 0.1982583	total: 22.4s	remaining: 51.7s
130:	learn: 0.1899634	total: 24.2s	remaining: 49.8s
140:	learn: 0.1822700	total: 26.1s	remaining: 47.9s
150:	learn: 0.1757999	total: 27.9s	remaining: 46s
160:	learn: 0.1698522	total: 29.7s	remaining: 44.2s
170:	learn: 0.1644116	total: 31.6s	remaining: 42.4s
180:	learn: 0.1598292	total: 33.5s	remaining: 40.5s
190:	learn: 0.1552037	tot

In [ ]:
predict_cat=cat.predict(features_test)

In [ ]:
f1_cat=f1_score(predict_cat,target_test)
f1_cat

0.627906976744186